# 1. Package 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

# 2. Knowledge Base 구성을 위한 데이터 생성

- 4. LangChain을 활용한 Vector Database 변경 (Chroma -> Pinecone)과 동일함

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 소득세법
# document_name = './tax_with_markdown.docx'
# 민법
# document_name = './civil_with_markdown.docx'
# 형법
# document_name = './criminal_with_markdown.docx'
# 도로교통법
document_name = './traffic_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len_list = len(document_list)
len_list

124

In [3]:
document_list[len_list - 10]

Document(metadata={'source': './traffic_with_markdown.docx'}, page_content='2. 제6조제1항ㆍ제2항ㆍ제4항 또는 제7조에 따른 금지ㆍ제한 또는 조치를 위반한 차 또는 노면전차의 운전자\n\n3. 제22조, 제23조, 제29조제4항부터 제6항까지, 제53조의5, 제60조, 제64조, 제65조 또는 제66조를 위반한 사람\n\n4. 제31조, 제34조 또는 제52조제4항을 위반하거나 제35조제1항에 따른 명령을 위반한 사람\n\n5. 제39조제6항에 따른 시ㆍ도경찰청장의 제한을 위반한 사람\n\n6. 제50조제1항, 제3항 및 제4항을 위반하여 좌석안전띠를 매지 아니하거나 인명보호 장구를 착용하지 아니한 운전자(자전거 운전자는 제외한다)\n\n6의2. 제56조의2제1항을 위반하여 자율주행시스템의 직접 운전 요구에 지체 없이 대응하지 아니한 자율주행자동차의 운전자\n\n7. 제95조제2항에 따른 경찰공무원의 운전면허증 회수를 거부하거나 방해한 사람\n\n8. 삭제<2020. 5. 26.>\n\n9. 삭제<2020. 5. 26.>\n\n9의2. 삭제<2020. 5. 26.>\n\n10. 주ㆍ정차된 차만 손괴한 것이 분명한 경우에 제54조제1항제2호에 따라 피해자에게 인적 사항을 제공하지 아니한 사람\n\n11. 제44조제1항을 위반하여 술에 취한 상태에서 자전거등을 운전한 사람\n\n12. 술에 취한 상태에 있다고 인정할 만한 상당한 이유가 있는 사람으로서 제44조제2항에 따른 경찰공무원의 측정에 응하지 아니한 사람(자전거등을 운전한 사람으로 한정한다)\n\n13. 제43조를 위반하여 제80조에 따른 원동기장치자전거를 운전할 수 있는 운전면허를 받지 아니하거나(원동기장치자전거를 운전할 수 있는 운전면허의 효력이 정지된 경우를 포함한다) 국제운전면허증 또는 상호인정외국면허증 중 원동기장치자전거를 운전할 수 있는 것으로 기재된 국제운전면허증 또는 상호인정외국면허증을 발급받지 아니하고(운전이 금지된 경우와 유효기

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
%pip install --upgrade jupyter

In [ ]:
%pip install --upgrade ipywidgets

In [5]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

is_create = False
# 소득세법
# index_name = 'tax-markdown-index'
# 민법
# index_name = 'civil-markdown-index'
# 형법
# index_name = 'criminal-markdown-index'
# 도로교통법
index_name = 'traffic-markdown-index'

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터를 추가할 때는 `from_documents()` 데이터를 추가한 이후에는 `from_existing_index()`를 사용합니다
if is_create:
    database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
else:
    database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

In [6]:
# 소득세법
# query = '연봉 5천만원인 직장인의 소득세는?'
# 민법
# query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'
# 형법
# query = '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?'
# 도로교통법
query = '음주운전 단속 기준은 어떻게 되나요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [7]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './traffic_with_markdown.docx'}, page_content='② 경찰공무원은 교통의 안전과 위험방지를 위하여 필요하다고 인정하거나 제1항을 위반하여 술에 취한 상태에서 자동차등, 노면전차 또는 자전거를 운전하였다고 인정할 만한 상당한 이유가 있는 경우에는 운전자가 술에 취하였는지를 호흡조사로 측정할 수 있다. 이 경우 운전자는 경찰공무원의 측정에 응하여야 한다.<개정 2014. 12. 30., 2018. 3. 27.>\n\n③ 제2항에 따른 측정 결과에 불복하는 운전자에 대하여는 그 운전자의 동의를 받아 혈액 채취 등의 방법으로 다시 측정할 수 있다.\n\n④ 제1항에 따라 운전이 금지되는 술에 취한 상태의 기준은 운전자의 혈중알코올농도가 0.03퍼센트 이상인 경우로 한다.<개정 2018. 12. 24.>\n\n⑤ 제2항 및 제3항에 따른 측정의 방법, 절차 등 필요한 사항은 행정안전부령으로 정한다.<신설 2023. 1. 3.>\n\n[전문개정 2011. 6. 8.]\n\n[시행일: 2024. 10. 25.] 제44조\n\n\n\n제45조(과로한 때 등의 운전 금지) 자동차등(개인형 이동장치는 제외한다) 또는 노면전차의 운전자는 제44조에 따른 술에 취한 상태 외에 과로, 질병 또는 약물(마약, 대마 및 향정신성의약품과 그 밖에 행정안전부령으로 정하는 것을 말한다. 이하 같다)의 영향과 그 밖의 사유로 정상적으로 운전하지 못할 우려가 있는 상태에서 자동차등 또는 노면전차를 운전하여서는 아니 된다. <개정 2013. 3. 23., 2014. 11. 19., 2017. 7. 26., 2018. 3. 27., 2020. 6. 9.>\n\n[전문개정 2011. 6. 8.]\n\n\n\n제46조(공동 위험행위의 금지) ① 자동차등(개인형 이동장치는 제외한다. 이하 이 조에서 같다)의 운전자는 도로에서 2명 이상이 공동으로 2대 이상의 자동차등을 정당한 사유 없이 앞뒤로 또는 좌우로 줄지어 통행

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [8]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
ai_message = qa_chain.invoke({"query": query})

In [12]:
ai_message

{'query': '음주운전 단속 기준은 어떻게 되나요?',
 'result': '음주운전 단속 기준은 운전자의 혈중알코올농도가 0.03퍼센트 이상인 경우입니다. 경찰공무원은 호흡조사로 혈중알코올농도를 측정할 수 있으며, 측정 결과에 불복할 경우 혈액 채취 등의 방법으로 다시 측정할 수 있습니다.'}